# LLM generation text detection 

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import keras
import keras_nlp
import re
import spacy
import time
import string


train_essays = pd.read_csv("/kaggle/input/llm-detect-ai-generated-text/train_essays.csv")
train_prompts = pd.read_csv("/kaggle/input/llm-detect-ai-generated-text/train_prompts.csv")
train_daigt = pd.read_csv("/kaggle/input/daigt-v2-train-dataset/train_v2_drcat_02.csv")




/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Using TensorFlow backend


## Datasets 

In [2]:
train_daigt["generated"] = train_daigt["label"]
train_essays = train_essays.merge(train_prompts, on='prompt_id', how='inner')

train_essays = pd.concat([train_essays,train_daigt])

#train_essays.loc[:,["prompt_name","generated"]].value_counts()
train_essays = train_essays.sample(frac = 1)

train_essays = train_essays.loc[:,["text","generated"]]


## Preprocessing

In [3]:
from multiprocessing import Pool

# Load the English NLP model
nlp = spacy.load("en_core_web_sm")

table = str.maketrans("", "", string.punctuation)
table[10] = None #\n
table[92] = None #\


#to lower
for code in range(26):
    table[code + 65] = code +97




def preprocess_text(text):
    
    #characters level preprocessing
    #remove \n and \, remove puntuactions, to lower case
    text = text.translate(table)
    
    # Tokenization using spaCy
    doc = nlp(text)
    
    # Remove stopwords and lemmization using spaCy's built-in stopword list
    tokens = [token.lemma_ for token in doc if not nlp.vocab[token.text].is_stop]
    
    return " ".join(tokens)




start_time = time.time()
# Number of parallel processes (adjust according to your CPU cores)
num_processes = 4

# Split the DataFrame into chunks
chunks = np.array_split(train_essays.loc[:,"text"], num_processes)

# Function to apply to each chunk in parallel
def parallel_map(chunk):
    return chunk.map(lambda x : preprocess_text(x))

# Initialize a Pool for parallel processing
with Pool(num_processes) as pool:
    # Use map function to apply the parallel_map function to each chunk
    results = pool.map(parallel_map, chunks)

# Concatenate the results back into a single DataFrame
train_essays["text"] = pd.concat(results, axis=0)

#train_essays.loc[:,"text"] = train_essays.loc[:,"text"].map(lambda x : preprocess_text(x))
# Convert back to pandas DataFrame (if needed)
end_time = time.time()

print(end_time - start_time)

/opt/conda/lib/python3.10/site-packages/numpy/core/fromnumeric.py:57: FutureWarning: 'Series.swapaxes' is deprecated and will be removed in a future version. Please use 'Series.transpose' instead.
  return bound(*args, **kwds)


1352.5375156402588


## BERT

In [4]:
import tensorflow as tf

from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

def bert_kernel_initializer(stddev=0.02):
    return keras.initializers.TruncatedNormal(stddev=stddev)

print(train_essays.columns)

X = train_essays["text"].values
y = train_essays["generated"].values



#X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Assuming your labels are 0 and 1
class_weights = compute_class_weight('balanced', classes=np.unique(y), y=y)

BATCH_SIZE = 64
SHUFFLE_BUFFER_SIZE = 100
hidden_size = 1024
dropout_prob = 0.3

train_dataset = tf.data.Dataset.from_tensor_slices((X, y))
#val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val))

train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
#val_dataset = val_dataset.batch(BATCH_SIZE)



# Preprocessor
preprocessor = keras_nlp.models.BertPreprocessor.from_preset("bert_large_en_uncased")

train_preprocessed = (
    train_dataset.map(preprocessor, tf.data.AUTOTUNE).cache().prefetch(tf.data.AUTOTUNE)
)
"""
val_preprocessed = (
    val_dataset.map(preprocessor, tf.data.AUTOTUNE).cache().prefetch(tf.data.AUTOTUNE)
)
"""

# Backbone
backbone = keras_nlp.models.BertBackbone.from_preset("bert_large_en_uncased")

backbone.summary()

backbone.trainable = False
inputs = backbone.input

pooled = backbone(inputs)["pooled_output"]

x = keras.layers.Dense(hidden_size)(pooled)
x = keras.layers.Dense(hidden_size, activation = 'relu')(x)
x = keras.layers.Dropout(dropout_prob)(x)
x = keras.layers.Dense(hidden_size, activation='relu')(x)
x = keras.layers.Dense(hidden_size,  activation='relu')(x)
x = keras.layers.Dropout(dropout_prob)(x)
x = keras.layers.Dense(hidden_size//8, activation = 'relu')(x)

outputs = keras.layers.Dense(1, activation='sigmoid')(x)


model = keras.Model(inputs, outputs)



Index(['text', 'generated'], dtype='object')


Attaching 'tokenizer.json' from model 'keras/bert/keras/bert_large_en_uncased/1' to your Kaggle notebook...
Attaching 'tokenizer.json' from model 'keras/bert/keras/bert_large_en_uncased/1' to your Kaggle notebook...
Attaching 'assets/tokenizer/vocabulary.txt' from model 'keras/bert/keras/bert_large_en_uncased/1' to your Kaggle notebook...
Attaching 'config.json' from model 'keras/bert/keras/bert_large_en_uncased/1' to your Kaggle notebook...
Attaching 'config.json' from model 'keras/bert/keras/bert_large_en_uncased/1' to your Kaggle notebook...
Attaching 'model.weights.h5' from model 'keras/bert/keras/bert_large_en_uncased/1' to your Kaggle notebook...


Model: "bert_backbone"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 token_ids (InputLayer)      [(None, None)]               0         []                            
                                                                                                  
 token_embedding (Reversibl  (None, None, 1024)           3125452   ['token_ids[0][0]']           
 eEmbedding)                                              8                                       
                                                                                                  
 segment_ids (InputLayer)    [(None, None)]               0         []                            
                                                                                                  
 position_embedding (Positi  (None, None, 1024)           524288    ['token_embedding[

## Training

In [5]:
"""
batch_sizes = [64,32]
learning_rates = [4e-4,3e-3,5e-5]


for batch_size in batch_sizes:
    
    train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
    val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val))
    
    train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(batch_size)
    val_dataset = val_dataset.batch(batch_size)




    train_preprocessed = (
        train_dataset.map(preprocessor, tf.data.AUTOTUNE).cache().prefetch(tf.data.AUTOTUNE)
    )
    val_preprocessed = (
        val_dataset.map(preprocessor, tf.data.AUTOTUNE).cache().prefetch(tf.data.AUTOTUNE)
    )
    for lr in learning_rates:
        print("EXECUTION BATCH SIZE {} LR {}".format(batch_size,lr))

        model.compile(
            loss=keras.losses.BinaryCrossentropy(from_logits=False),
            optimizer=keras.optimizers.AdamW(lr),
            metrics = [keras.metrics.AUC()],
            jit_compile=True,
        )

        model.fit(
            train_preprocessed,
            validation_data=val_preprocessed,
            epochs=2,
            class_weight = {0:class_weights[0],1:class_weights[1]}
        )
"""

model.compile(
            loss=keras.losses.BinaryCrossentropy(from_logits=False),
            optimizer=keras.optimizers.AdamW(2e-4),
            metrics = [keras.metrics.AUC()],
            jit_compile=True,
        )

model.fit(
    train_preprocessed,
    epochs=10,
    class_weight = {0:class_weights[0],1:class_weights[1]}
)

Epoch 1/10
723/723 [==============================] - 2447s 3s/step - loss: 0.6682 - auc: 0.6246
Epoch 2/10
723/723 [==============================] - 2349s 3s/step - loss: 0.5638 - auc: 0.7793
Epoch 3/10
723/723 [==============================] - 2348s 3s/step - loss: 0.5243 - auc: 0.8155
Epoch 4/10
723/723 [==============================] - 2349s 3s/step - loss: 0.4982 - auc: 0.8361
Epoch 5/10
723/723 [==============================] - 2349s 3s/step - loss: 0.4919 - auc: 0.8412
Epoch 6/10
723/723 [==============================] - 2350s 3s/step - loss: 0.4725 - auc: 0.8553
Epoch 7/10
723/723 [==============================] - 2349s 3s/step - loss: 0.4673 - auc: 0.8588
Epoch 8/10
723/723 [==============================] - 2349s 3s/step - loss: 0.4624 - auc: 0.8620
Epoch 9/10
723/723 [==============================] - 2349s 3s/step - loss: 0.4549 - auc: 0.8670
Epoch 10/10
723/723 [==============================] - 2349s 3s/step - loss: 0.4539 - auc: 0.8675


## Evaluation and submission

In [6]:
test_essays = pd.read_csv("/kaggle/input/llm-detect-ai-generated-text/test_essays.csv")
test_essays.loc[:,"text"] = test_essays.loc[:,"text"].map(lambda x : preprocess_text(x))
print(preprocessor(test_essays["text"]))
submission_dict  = pd.DataFrame()
submission_dict["id"] = test_essays["id"]
submission_dict["generated"] = model.predict(preprocessor(test_essays["text"]))[:,0]


submission = pd.DataFrame.from_dict(submission_dict)
submission.to_csv("/kaggle/working/submission.csv",index=False)

{'token_ids': <tf.Tensor: shape=(3, 512), dtype=int32, numpy=
array([[  101, 13360, 22861, ...,     0,     0,     0],
       [  101, 22861,  2497, ...,     0,     0,     0],
       [  101, 10507,  2278, ...,     0,     0,     0]], dtype=int32)>, 'segment_ids': <tf.Tensor: shape=(3, 512), dtype=int32, numpy=
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int32)>, 'padding_mask': <tf.Tensor: shape=(3, 512), dtype=bool, numpy=
array([[ True,  True,  True, ..., False, False, False],
       [ True,  True,  True, ..., False, False, False],
       [ True,  True,  True, ..., False, False, False]])>}
1/1 [==============================] - 10s 10s/step
